In [2]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [17]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for batch, (X, y) in enumerate(train_dataloader):
  print(f"Batch {batch}")
  print(f"X_shape: {X.shape}")
  print(f"y_shape: {y.shape}")
  break

Batch 0
X_shape: torch.Size([64, 1, 28, 28])
y_shape: torch.Size([64])


In [5]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device}")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.seq = nn.Sequential(
        nn.Linear(28 * 28, 256), # Input layer
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 10) # Output layer
    )

  def forward(self, X):
    X = self.flatten(X)
    return self.seq(X)

model = NeuralNetwork().to(device)
model

Using cpu


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (seq): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # print(f"Batch: {batch}")
    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.202591  [   64/60000]
loss: 0.308743  [ 6464/60000]
loss: 0.211863  [12864/60000]
loss: 0.341256  [19264/60000]
loss: 0.284628  [25664/60000]
loss: 0.322744  [32064/60000]
loss: 0.314107  [38464/60000]
loss: 0.415893  [44864/60000]
loss: 0.406375  [51264/60000]
loss: 0.355546  [57664/60000]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.370295 

Epoch 2
-------------------------------
loss: 0.200230  [   64/60000]
loss: 0.304949  [ 6464/60000]
loss: 0.209893  [12864/60000]
loss: 0.336565  [19264/60000]
loss: 0.283119  [25664/60000]
loss: 0.323610  [32064/60000]
loss: 0.311492  [38464/60000]
loss: 0.411019  [44864/60000]
loss: 0.399190  [51264/60000]
loss: 0.354578  [57664/60000]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.368442 

Epoch 3
-------------------------------
loss: 0.199143  [   64/60000]
loss: 0.300753  [ 6464/60000]
loss: 0.207541  [12864/60000]
loss: 0.332614  [19264/60000]
loss: 0.281033  [25664/60000]
loss: 0.317786  [32064/600